In [1]:
from vnstock import *
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [4]:
#MODEL
list_i = []
list_return_price = []
list_rmse, list_mse, list_mae = [], [], []
list_k = []
list_r2_train = []
list_r2_test = []

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    X = data.loc[:, ~data.columns.isin(['Today'])]
    y = data["Today"]

    list_k1=[]
    list_score=[]
    for k in np.arange(0.1,0.5,0.01):
      model = Lasso(alpha=1.0)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = k, random_state = 12, shuffle=False)
      model.fit(X_train, y_train)
      yhat_test = model.predict(X_test)
      score = model.score(X_test, y_test)
      list_k1.append(k)
      list_score.append(score)

    score_max = max(list_score)
    a = list_score.index(score_max)
    k = list_k1[a]

    model = Lasso(alpha=1.0)
    X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=k,random_state=12, shuffle=False)
    model.fit(X_train,y_train)
    yhat_test = model.predict(X_test)
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, yhat_test, squared=True)
    rmse = mean_squared_error(y_test, yhat_test, squared=False)
    mae = mean_absolute_error(y_test, yhat_test)

    X_predict = [X.iloc[-1].values.tolist()]
    Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_k.append(k)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {rmse:.2f}',f' MSE: {mse:.2f}', f'MAE: {mae:.2f}',f' test size: {k:.2f}',)
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -1.11  R2_train: 0.44  R2_test: 0.52  RMSE: 1.33  MSE: 1.77 MAE: 0.99  test size: 0.15
Stock: BCM  Predict: -0.51  R2_train: 0.27  R2_test: 0.40  RMSE: 0.92  MSE: 0.85 MAE: 0.66  test size: 0.25
Stock: VHM  Predict: 0.38  R2_train: 0.23  R2_test: 0.24  RMSE: 1.76  MSE: 3.11 MAE: 1.20  test size: 0.13
Stock: VIC  Predict: 0.24  R2_train: 0.18  R2_test: 0.15  RMSE: 1.87  MSE: 3.51 MAE: 1.03  test size: 0.21
Stock: VRE  Predict: -0.70  R2_train: 0.34  R2_test: 0.50  RMSE: 1.07  MSE: 1.15 MAE: 0.74  test size: 0.15
Stock: BVH  Predict: 0.66  R2_train: 0.34  R2_test: 0.57  RMSE: 0.76  MSE: 0.57 MAE: 0.49  test size: 0.18
Stock: POW  Predict: 0.90  R2_train: 0.38  R2_test: 0.52  RMSE: 1.13  MSE: 1.28 MAE: 0.80  test size: 0.21
Stock: GAS  Predict: 0.67  R2_train: 0.34  R2_test: 0.63  RMSE: 0.63  MSE: 0.39 MAE: 0.47  test size: 0.19
Stock: ACB  Predict: 0.47  R2_train: 0.24  R2_test: 0.36  RMSE: 0.82  MSE: 0.67 MAE: 0.48  test size: 0.15
Stock: BID  Predict: 0.70  R2_trai

In [5]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test,
                    'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,'test size': list_k})
KQ.to_excel("Kết quả linh hoạt test size.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE,test size
0,SSI,[-1.1143956992183717],0.435188,0.523579,1.768644,1.329904,0.993686,0.15
1,BCM,[-0.5091245675419191],0.266676,0.398628,0.848581,0.921185,0.655485,0.25
2,VHM,[0.38057290279422534],0.228588,0.235930,3.105186,1.762154,1.195741,0.13
3,VIC,[0.23550641999953434],0.183096,0.146022,3.508339,1.873056,1.029237,0.21
4,VRE,[-0.6953925220190632],0.337403,0.498116,1.151672,1.073160,0.743180,0.15
...,...,...,...,...,...,...,...,...
389,TDP,[0.4604719002621553],0.220045,0.314136,4.791199,2.188881,1.520926,0.49
390,NO1,[-0.2241271949425839],0.120308,0.081272,6.967199,2.639545,1.803919,0.44
391,L10,[2.165055239487991],0.525984,0.549506,13.548236,3.680793,3.049042,0.13
392,REE,[0.7462387675242582],0.298384,0.478536,0.826745,0.909255,0.659721,0.19
